# Comparing multiple CRNS probes

**For the time being, this notebook is a sandbox.** Step by step, we will migrate mature functions to the actual `cosmicsense` library.

For a detailed explanation of prepprocessing steps, please refer to [from_n_to_theta.ipynb](https://cosmicsense.readthedocs.io/en/latest/notebooks/n_to_theta.html).

In [1]:
# Standard packages
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as colors
import matplotlib as mpl
import numpy as np
import pandas as pd
import os.path as path
import datetime as dt
import warnings

from bokeh.plotting import figure, show, save, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import LinearAxis, HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.palettes import Spectral3
import bokeh.palettes
from bokeh.embed import components
from bokeh.layouts import gridplot
from bokeh.models import Range1d

import seaborn as sb

import cosmicsense as cs

In [2]:
warnings.simplefilter('once', RuntimeWarning)

In [3]:
# Display figures inline
%matplotlib inline
# Display figures as interactive (requires kernel restart)
#%matplotlib

Using matplotlib backend: Qt5Agg


## Read raw data

#### CRNS records from three probes with different temporal coverage

In [4]:
#ids = [1,2,3,4,16,17,18,19,23,24]
ids = [1, 2, 3, 4, 5, 6, 7, 8, 14, 16, 17, 18, 19, 21, 22, 23, 24, 25]

#fpath = "/home/maik/b2drop/cosmicsense/inbox/fendt/timeseries/crns/JFC-1/"
fpath = "/media/x/cosmicsense/data/fendt/crns/"
htmlfile = "/media/x/cosmicsense/data/fendt/crns/crnsmerged.html"
plot_width = 550

In [5]:
crns = {}
for id in ids:
    df = pd.read_csv(path.join(fpath, "%d/%d_CRNS_merge.txt" % (id, id)), sep="\t")
    df.datetime = pd.to_datetime(df.datetime)
    df = df.set_index("datetime")
    if id==4:
        df["cph1"] = (df.counts1 + df.counts2) / cs.conv.s_to_h(df.nsecs1)
    else:
        df["cph1"] = df.counts1 / cs.conv.s_to_h(df.nsecs1)
        try:
            df["cph2"] = df.counts2 / cs.conv.s_to_h(df.nsecs2)
        except AttributeError:
            pass
    print(id, end=": ")
    print("%s to %s" % (df.index[0], df.index[-1]) )
    crns[id] = df

1: 2019-05-09 09:59:00 to 2019-06-13 07:19:00
2: 2019-05-07 11:02:13 to 2019-06-13 05:40:00
3: 2019-05-07 08:37:25 to 2019-06-13 09:26:00
4: 2019-05-07 15:21:29 to 2019-06-13 05:39:00
5: 2019-05-03 08:53:04 to 2019-06-07 08:37:00
6: 2019-05-03 09:34:48 to 2019-06-07 08:41:00
7: 2019-05-13 14:54:00 to 2019-06-07 09:40:00
8: 2019-05-01 00:46:00 to 2019-06-07 08:46:00
14: 2019-05-08 06:45:44 to 2019-05-31 09:19:00
16: 2019-05-14 13:45:00 to 2019-06-12 16:31:00
17: 2019-05-15 14:14:46 to 2019-06-12 14:26:00
18: 2019-05-14 10:16:06 to 2019-06-06 14:35:00
19: 2019-05-14 12:45:49 to 2019-06-13 03:00:00
21: 2019-05-13 13:38:38 to 2019-06-10 20:56:00
22: 2019-05-13 15:24:44 to 2019-06-10 21:18:00
23: 2019-05-15 15:55:32 to 2019-06-13 04:19:00
24: 2019-05-15 15:09:58 to 2019-06-12 15:19:00
25: 2019-05-14 10:02:00 to 2019-06-07 08:41:00


In [6]:
min_dtime = np.min([crns[key].index[0] for key in crns.keys()])
max_dtime = np.max([crns[key].index[-1] for key in crns.keys()])

## Filter spurious signals

Some probes are affected by spurious count rates, presumably due to excess voltage from the solar panels in cases of intense insolation. In order to keep useful parts of the signal, a pragmatic/heuristic filtering approach is applied which could be refined later:

1. Remove entirely unrealisticly count rates for specific probes (`mincph`, `maxcph`)
2. Remove count rates from spuriously short count intervals (`mininterv`)
3. After that, there are still spurious count rates.In order to detect these, we compute the maximum count rates over periods of six hours, and then apply a 24-hour-median filter to these 6-hour-maxima. That way, we try to truncate spurios peaks. In order to prevent too aggressive filtering, we add a `buffer` of the median count rates over a period of 24 hours.
4. We then interpolate this upper limit filter to the original timestamp values and use it to remove high values.
5. We remove unrealisticly low count rates (`mincph`), and than apply the same approach (points 3-4) to eliminate spuriously small values.  

In [ ]:
pars =  {
    1: {"mincph": 500, "maxcph": 1000, "mininterv": -9999, "type": "CRS 2000-B, Scn.", "lut": "forest/meadow"},
    2: {"mincph": 500, "maxcph": 1100, "mininterv": -9999, "type": "CRS 1000", "lut": "meadow"},
    3: {"mincph": 500, "maxcph": 1100, "mininterv": -9999, "type": "CRS 1000", "lut": "meadow"},
    4: {"mincph": 6000, "maxcph": 9500, "mininterv": -9999, "type": "Lab C", "lut": "meadow"},
    5: {"mincph": 800, "maxcph": 1500, "mininterv": -9999, "type": "CRS 1000-B", "lut": "meadow"},
    6: {"mincph": 800, "maxcph": 1500, "mininterv": -9999, "type": "CRS 1000-B", "lut": "meadow, forest close"},
    7: {"mincph": 1000, "maxcph": 1700, "mininterv": -9999, "type": "CRS 1000-B", "lut": "meadow"},
    8: {"mincph": 1300, "maxcph": 2500, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow"},
    14: {"mincph": 800, "maxcph": 1500, "mininterv": -9999, "type": "CRS 2000", "lut": "forest"},
    16: {"mincph": 1500, "maxcph": 2500, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow"},
    17: {"mincph": 1400, "maxcph": 2400, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow"},
    18: {"mincph": 500, "maxcph": 1000, "mininterv": -9999, "type": "CRS 1000", "lut": "meadow"},
    19: {"mincph": 1300, "maxcph": 2300, "mininterv": -9999, "type": "CRS 2000-B", "lut": "forest"},
    21: {"mincph": 1400, "maxcph": 2300, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow, forest close"},
    22: {"mincph": 1100, "maxcph": 2100, "mininterv": -9999, "type": "CRS 2000-B", "lut": "forest"},
    23: {"mincph": 1200, "maxcph": 2200, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow, peat"},
    24: {"mincph": 1600, "maxcph": 2600, "mininterv": -9999, "type": "CRS 2000-B", "lut": "meadow"},
    25: {"mincph": 900, "maxcph": 1500, "mininterv": -9999, "type": "CRS 1000-B", "lut": "meadow"},
    
}

buffer = 0.075

for i, key in enumerate(crns.keys()):
    x = crns[key].cph1.copy()
    if not key==1:
        x[x > pars[key]["maxcph"]] = np.nan
    x[x < pars[key]["mincph"]] = np.nan
    x[crns[key].nsecs1 < pars[key]["mininterv"]] = np.nan
    median24 = x.resample("24H").median()
    # Maxfilter
    max6 = x.resample("6H").max()
    median24max6 = max6.resample("24H").median()
    maxfilter = np.array(median24max6 + buffer * median24)
    # Minfilter
    min6 = x.resample("6H").min()
    median24min6 = min6.resample("24H").median()
    minfilter = np.array(median24min6 - buffer * median24)    
    # Resample filter to original time stamps
    crns[key]["cph1_maxfilter"] = np.interp(x.index, median24.index, maxfilter)
    crns[key]["cph1_minfilter"] = np.interp(x.index, median24.index, minfilter)
    # Fill gaps
    crns[key]["cph1_maxfilter"] = crns[key].cph1_maxfilter.interpolate()
    crns[key]["cph1_minfilter"] = crns[key].cph1_minfilter.interpolate()
    # Apply filter
    crns[key]["cph1_filtered"] = x
    if not key==1:
        crns[key].loc[crns[key].cph1 > crns[key].cph1_maxfilter, "cph1_filtered"] = np.nan
        crns[key].loc[crns[key].cph1 < crns[key].cph1_minfilter, "cph1_filtered"] = np.nan

In [ ]:
plt.rc('font', **{'size'   : 12})
titles = ["CRNS #1", "CRNS #2", "CRNS #3", "NMDB station counts", "Barometric pressure"]
fig, ax = plt.subplots(nrows=len(crns), figsize=(12,40))

xlim = min_dtime, max_dtime

for i, key in enumerate(crns.keys()):
    ax[i].plot(crns[key].index, crns[key].cph1, linestyle="None", marker=".", ms=1, color="red")
    ax[i].plot(crns[key].index, crns[key].cph1_maxfilter, linestyle="-", ms=0, color="orange")
    ax[i].plot(crns[key].index, crns[key].cph1_minfilter, linestyle="-", ms=0, color="orange")
    ax[i].plot(crns[key].index, crns[key].cph1_filtered, linestyle="None", marker=".", ms=1, color="black")
    ax[i].set_title(key)
    ax[i].set_xlabel("")
    ax[i].set_ylabel("cph")
    ax[i].set_xlim(xlim)
    #ax[i].set_ylim(pars[key]["mincph"], pars[key]["maxcph"]+200)
    ax[i].grid()
    #ax[i].legend()    
    
plt.tight_layout()

## Resample to a common time interval

In [ ]:
min_dtime, max_dtime

In [ ]:
dtrange6 = pd.date_range('2019-05-01 00:00:00', max_dtime, freq="6H")
dtrange24 = pd.date_range('2019-05-01 00:00:00', max_dtime, freq="24H")
crns6h = pd.DataFrame({}, index=dtrange6)
crns24h = pd.DataFrame({}, index=dtrange24)

for i, key in enumerate(crns.keys()):
    crns6h = pd.merge(crns6h, crns[key].cph1_filtered.resample("6H").mean(), 
                      how="left", left_index=True, right_index=True)
    crns6h[key] = crns6h.cph1_filtered
    crns6h = crns6h.drop("cph1_filtered", axis=1)
    # 24 h
    crns24h = pd.merge(crns24h, crns[key].cph1_filtered.resample("24H").mean(), 
                      how="left", left_index=True, right_index=True)
    crns24h[key] = crns24h.cph1_filtered
    crns24h = crns24h.drop("cph1_filtered", axis=1)

In [ ]:
htmlfile_merge = "/media/x/cosmicsense/git/misc/fendt/merged_crns.html"
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
colors=bokeh.palettes.Category20_20#bokeh.palettes.Set3_12 + bokeh.palettes.Set2_6

plts = []

from bokeh.models import Span
from bokeh.models import Label

for i, id in enumerate(ids):
    p = figure(title="Counts/hour (moder.), #%d (%s, %s)" % (id, pars[id]["type"], pars[id]["lut"]), x_axis_type='datetime',
               y_axis_type="linear", plot_height = 200, plot_width = plot_width,
               tools = TOOLS)
    if id==1:
        vline = Span(location=dt.datetime(2019,6,6,8,0,0), dimension='height', line_color='red', line_width=1, line_dash=[2])
        p.renderers.extend([vline])
        mytext = Label(x=dt.datetime(2019,6,6,8,0,0), y=1200, text='Shield rem.',
                       text_align="right", text_font_size="9pt")
        p.add_layout(mytext)
    p.xaxis.axis_label = 'Datetime'
    p.yaxis.axis_label = 'Counts per hour'
    p.circle(crns[id].index, crns[id].cph1_filtered, size=1, color="lightgrey", legend="20 mins")
    p.line(crns6h.index+dt.timedelta(hours=3), crns6h[id], line_color="black", line_width=2, legend="6 hours")
    #p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id], line_color="black", line_width=2, legend="6 hours")
    #p.line(crns24h.index+dt.timedelta(hours=12), crns24h[id], line_color="green", line_dash=[2], line_width=2, legend="24 hours")
    p.x_range=Range1d(min_dtime, max_dtime)
    miny, maxy = None, None
    if pars[id]["type"]=="CRS 2000-B":
        miny, maxy = 1400, 2400
    if pars[id]["type"]=="CRS 1000":
        miny, maxy = 500, 1000
    if pars[id]["type"]=="CRS 1000-B":
        miny, maxy = 900, 1500
    if pars[id]["type"]=="CRS 2000":
        miny, maxy = 1000, 1400
    if miny is not None:    
        p.y_range=Range1d(miny, maxy)
    p.legend.location = "center_left"
    p.legend.background_fill_alpha = 0.6
    #p.legend.location = "center_left"
    plts.append(p)


grid = gridplot(plts, merge_tools=False, ncols=1)

output_file(htmlfile_merge, title="Time series of counts per hour")
#save(grid)
show(grid)

# Add correct header to html
f = open(htmlfile_merge, "r")
html = f.read()
f.close()
html = html.strip()
replacement = '''<html>
<div class="blurb">
	<h1>JFC Fendt: Processed neutron counts  </h1>
</div><!-- /.blurb -->
<p> Data merged from SD and telemetry...counts per hour...filtered...resampled. 
'''
html = html.replace('<html lang="en">', replacement)
f = open(htmlfile_merge, "w")
f.write("---\nlayout: default\ntitle: cosmic pages\n---\n")
f.write(html)
f.close()

In [ ]:
sb.pairplot(crns6h)#, plot_kws=dict(edgecolor="None", s=6))

## Correcting for variations in incoming neutron flux

#### NMBD station data

`nmdb.txt` contains reference (background) neutron count rates from [NMDB](http://www.nmdb.eu/nest/), for stations `KIEL2`, `JUNG`, `JUNG1`, and `DRBS` (Dourbes, Belgium). 

In [ ]:
# NMDB data
nmdb = pd.read_csv("/media/x/cosmicsense/data/fendt/nmdb/nmdb.txt", sep=";", 
                   comment="#", na_values="   null")
nmdb.datetime = pd.to_datetime(nmdb.datetime)
nmdb = nmdb.set_index("datetime")

ax = nmdb.plot(figsize=(14,5), grid=True, ylim=(100,500))
ax.set_xlabel("")
ax.set_ylabel("cps")

In [ ]:
ax1 = plt.subplot()
nmdb.JUNG.plot(legend="JUNG")
plt.legend()
ax2 = ax1.twinx()
nmdb.JUNG1.plot(color="red", legend="JUNG1")
leg = plt.legend(loc="lower right")

Several literature references (e.g. Zreda et al. 2012, Schroen et al. 2015, Andreasen et al. 2017) suggest to correct for variations in incoming neutron fluxes based on cosmic-ray neutron monitors available through http://www.nmdb.eu/nest. The idea is to compute a simple scaling factor $f_i$ based on measure neutron intensity $f_m$ and an arbitrary reference intensity $f_{ref}$ that depends on the actual neutron monitor.

\begin{equation*}
f_i = \frac{I_m}{I_{ref}}
\end{equation*}

In the dissertation thesis of Schroen (2016), a reference value $f_{ref}$ of 150 cps is suggested for the monitor on Jungfraujoch (JUNG).

The following figure shows the time series of our CRNS counts and the NMDB data.

In [ ]:
fi = (nmdb.JUNG / 150.).resample("6H").mean()
fi.name="fi"
fi.plot()

## Correcting for variations in barometric pressure

Again based on Zreda et al. (2012), Andreasen et al. (2017) and many others, a correction factor $f_p$ is suggested in order to account for variations in barometric pressure.

\begin{equation*}
f_p = exp\Bigl(\frac{p_0 - p}{L}\Bigl)
\end{equation*}

Quoting from [Andreasen et al. (2017)](https://dl.sciencesocieties.org/publications/vzj/pdfs/16/8/vzj2017.04.0086):

> [...] $L$  is  the  mass  attenuation  length  for  high-energy  neutrons and is a function of cutoff rigidity
> (Desilets et al., 2006), $p$ is the barometric pressure at the time of measurement, and $P_0$ is an arbitrary
> reference pressure. Note that the units of $L$, $p$, and $p_0$ can be shielding depth (g/cm2) or pressure (Pa), 
> where $1 g/cm2 = 98.0665 Pa$. If shielding depth is used, $L$ ranges from 130 g/cm2 at high latitudes to 
> 144 g/cm2 at the equator (see Fig. 1).

[Zreda et al. (2012)](https://www.hydrol-earth-syst-sci.net/16/4079/2012/hess-16-4079-2012.pdf) complement that

> [... $p_0$] can be selected to be the long-term average pressure at the specific site, sea-level pressure, 
> or long-term averagepressure at a different reference site.

**Question: How do we quantify $p_0$? Based on site average, or just based on standard sea level pressure (1013 mbar)?**

For now, we use $p_0 = 1013.25 mbar = 101325 Pa = 1033.23 g/cm²$ and $L=131.6$ for Germany (Fig. 1 in Andreasen et al. (2017).

In [ ]:
f_press = "/media/x/cosmicsense/data/fendt/dwd/stundenwerte_P0_02290_akt/produkt_p0_stunde_20171208_20190610_02290.txt"
press = pd.read_csv(f_press, sep=";", na_values=-999)
press.columns = ["station_id", "datetime", "quality", "p", "p0", "eor"]
press.datetime = pd.to_datetime(press.datetime, format="%Y%m%d%H")
press = press.set_index("datetime")

In [ ]:
plt.plot(press.p0, cs.core.corrfact_baro(np.array(press.p0), p_0, L))
plt.xlabel("Pressure (mbar)")
plt.ylabel("Inverse correction factor")

In [ ]:
p_0 = press.p0.mean()
L = 131.6 # g/cm2
fp = cs.core.corrfact_baro(np.array(press.p0), p_0, L)
fp = pd.Series(fp, index=press.index)
fp = fp.resample("6H").mean()
fp.name="fp"
fp["2019-05-01":].plot()

## Correcting for variations in atmospheric water vapor

In their overview, Andreasen et al. (2017) refer to Rosolem et al. (2013) when accounting for the effects of atmospheric water vapor:

\begin{equation*}
f_{wv} = 1 + 0.0054 * (h - h_{ref})
\end{equation*}

where $h$ is the absolute humidity of the air (in g/m3), and $h_{ref}$ is the absolute humidity at an arbitrary reference time.

The references do not elaborate on how to obtain the absolute humidity, but given the relative humidity and air temperature, we typically obtain $h$ by combining 

1. Relationship between vapor pressure $e$, saturated vapor pressure $e_s$ and relative humidity $rh$ (in %)

\begin{equation*}
e = e_s * rh / 100.
\end{equation*}

2. August-Roche-Magnus approximation of relation betweeen $e_s$ (mbar) and air temperature $T$ (in deg C) 

\begin{equation*}
e_s(T) = 6.1094 * exp\Bigl(\frac{17.625*T}{T + 243.04}\Bigl)
\end{equation*}

3. Universal law of perfect gases (with volume $V$, mass $m$, specific gas constant $R_S=461.4 J/kg/K$ for water vapor)

\begin{equation*}
e * V = m * R_s * T
\end{equation*}

In [ ]:
# dummy correction factor for humidity
fwv = pd.Series(data=np.ones(len(crns6h.index)), index=crns6h.index)
fwv.name = "fwv"

## Combining the correction factors

We can now inspect the different correction factor, and use them to correct our neutron counts. According to Andreasen, this is done via

\begin{equation*}
N_{cor} = \frac{N*f_{wv}}{f_p*f_i}
\end{equation*}

In [ ]:
crns6hc = crns6h.copy()
crns6hc = pd.merge(crns6hc, fi, how='left', left_index=True, right_index=True)
crns6hc = pd.merge(crns6hc, fp, how='left', left_index=True, right_index=True)
crns6hc = pd.merge(crns6hc, fwv, how='left', left_index=True, right_index=True)
#crns6hc = crns6hc.rename(index=str, columns={"JUNG1": "fi", "p0": "fp"})
#crns6hc.index = pd.to_datetime(crns6hc.index)

In [ ]:
#crns6hc = crns6h.copy()
for id in ids:
    crns6hc[id] = crns6hc[id] * crns6hc["fwv"] / (crns6hc["fi"] * crns6hc["fp"])

In [ ]:
f_prec = "/media/x/cosmicsense/data/fendt/dwd/stundenwerte_RR_02290_akt/produkt_rr_stunde_20171208_20190610_02290.txt"
prec = pd.read_csv(f_prec, sep=";", na_values=-999)

prec.columns = ["station_id", "datetime", "quality", "depth", "ind", "wrtr", "eor"]
prec.datetime = pd.to_datetime(prec.datetime, format="%Y%m%d%H")
prec = prec.set_index("datetime")
prec["2019-05-06":].depth.cumsum().plot()

In [ ]:
crns6hcst = crns6hc.copy()
#crns6hcst = crns6hcst.drop(columns=["fi", "fp", "fwv"])
for i, id in enumerate(ids):
    if id==19:
        fact = np.nanmean(crns6hc["2019-05-22":"2019-05-29"][4]) / np.nanmean(crns6hc["2019-05-22":"2019-05-29"][id])
        crns6hcst[id] = crns6hc[id] * fact
    elif id==1:
        #continue
        fact2 = np.nanmean(crns6hc["2019-06-06 12:00:00":"2019-06-12 00:00:00"][1]) / np.nanmean(crns6hc["2019-05-22":"2019-05-29"][1])
        crns1 = crns6hc[1].copy()
        crns1[:"2019-06-06 03:00:00"] *= fact2
        fact = np.nanmean(crns6hc["2019-05-22":"2019-05-29"][4]) / np.nanmean(crns1["2019-05-22":"2019-05-29"])
        crns1 *= fact
        crns6hcst[id] = crns1
    else:
        fact = np.nanmean(crns6hc["2019-05-15":"2019-05-22"][4]) / np.nanmean(crns6hc["2019-05-15":"2019-05-22"][id])
        crns6hcst[id] = crns6hc[id] * fact

In [ ]:
crns6hcst.index+dt.timedelta(hours=3)

In [ ]:
plt.rc('font', **{'size'   : 12})
colors = plt.cm.tab20(np.linspace(0,1,len(ids)))
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(311)
crns24chst = crns6hcst.resample("24H").mean()
#pl = crns6hcst.resample("24H").mean().plot(y=ids, ax=ax1, color=colors)
#crns6hcst.plot(ax=ax1, color=colors)
for i, id in enumerate(ids):
    if id==1:
        continue
    ax1.plot(crns6hcst.index+dt.timedelta(hours=3), crns6hcst[id], color=colors[i])
    #ax1.plot(crns24chst.index+dt.timedelta(hours=12), crns24chst[id], color=colors[i])
ax1.legend(ncol=2)
plt.ylim(6500, 8000)
plt.grid()
ax2 = fig.add_subplot(312)
ax2.plot(prec["2019-05-01":].index, prec["2019-05-01":].depth.cumsum())
#prec[crns6hcst.index[0]:crns6hcst.index[-1]].depth.cumsum().plot()
ax2.set_xlim(ax1.get_xlim())
plt.grid()
ax3 = fig.add_subplot(313)
ax3.plot(fi.index, fi, label="fi")
ax3.plot(fp.index, fp, label="fp")
ax3.set_xlim(ax1.get_xlim())
#crns6hcst.fi.plot()
#crns6hcst.fp.plot()
ax3.legend()
plt.grid()
plt.tight_layout()


In [ ]:
crns6hcst[4].plot()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(crns[4].cph1.index, crns[4].cph1, "bo", ms=1)
plt.plot(crns6h.index + dt.timedelta(hours=3), crns6h[4])
plt.grid()
plt.ylim(6000, 9000)


In [ ]:
plt.cm.tab20(18)

In [ ]:
htmlfile_merge = "/media/x/cosmicsense/git/misc/fendt/merged_crns.html"
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
colors=bokeh.palettes.Category20_20#bokeh.palettes.Set3_12 + bokeh.palettes.Set2_6

plts = []

from bokeh.models import Span
from bokeh.models import Label

for i, id in enumerate(ids):
    p = figure(title="Counts/hour (moder.), #%d (%s, %s)" % (id, pars[id]["type"], pars[id]["lut"]), x_axis_type='datetime',
               y_axis_type="linear", plot_height = 200, plot_width = plot_width,
               tools = TOOLS)
    if id==1:
        vline = Span(location=dt.datetime(2019,6,6,8,0,0), dimension='height', line_color='red', line_width=1, line_dash=[2])
        p.renderers.extend([vline])
        mytext = Label(x=dt.datetime(2019,6,6,8,0,0), y=1200, text='Shield rem.',
                       text_align="right", text_font_size="9pt")
        p.add_layout(mytext)
    p.xaxis.axis_label = 'Datetime'
    p.yaxis.axis_label = 'Counts per hour'
    #p.circle(crns[id].index, crns[id].cph1_filtered, size=1, color="lightgrey", legend="20 mins")
    p.line(crns6h.index+dt.timedelta(hours=3), crns6h[id], line_color="grey", line_width=2, legend="uncorrected")
    p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id], line_color="black", line_width=2, legend="corrected")
    p.line(prec["2019-05-06":].index, prec["2019-05-06":].depth.cumsum(), line_color="blue", line_width=2, legend="precipitation")
    #p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id], line_color="black", line_width=2, legend="6 hours")
    #p.line(crns24h.index+dt.timedelta(hours=12), crns24h[id], line_color="green", line_dash=[2], line_width=2, legend="24 hours")
    p.x_range=Range1d(min_dtime, max_dtime)
    miny, maxy = None, None
    if pars[id]["type"]=="CRS 2000-B":
        miny, maxy = 1400, 2400
    if pars[id]["type"]=="CRS 1000":
        miny, maxy = 500, 1000
    if pars[id]["type"]=="CRS 1000-B":
        miny, maxy = 900, 1500
    if pars[id]["type"]=="CRS 2000":
        miny, maxy = 1000, 1400
    if miny is not None:    
        p.y_range=Range1d(miny, maxy)
    p.legend.location = "center_left"
    p.legend.background_fill_alpha = 0.6
    #p.legend.location = "center_left"
    plts.append(p)


grid = gridplot(plts, merge_tools=False, ncols=1)

output_file(htmlfile_merge, title="Time series of counts per hour")
#save(grid)
show(grid)

# Add correct header to html
f = open(htmlfile_merge, "r")
html = f.read()
f.close()
html = html.strip()
replacement = '''<html>
<div class="blurb">
	<h1>JFC Fendt: Processed neutron counts  </h1>
</div><!-- /.blurb -->
<p> Data merged from SD and telemetry...counts per hour...filtered...resampled. 
'''
html = html.replace('<html lang="en">', replacement)
f = open(htmlfile_merge, "w")
f.write("---\nlayout: default\ntitle: cosmic pages\n---\n")
f.write(html)
f.close()

In [ ]:
htmlfile_merge = "/media/x/cosmicsense/git/misc/fendt/merged_crns.html"
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
colors=bokeh.palettes.Category20_20#bokeh.palettes.Set3_12 + bokeh.palettes.Set2_6

plts = []

from bokeh.models import Span
from bokeh.models import Label

p = figure(title="Counts/hour (moder.), norm. to CRNS #4; cum. prec. (H.-Peiss.)", x_axis_type='datetime',
           y_axis_type="linear", plot_height = 800, plot_width = 800,#plot_width,
           tools = TOOLS)

for i, id in enumerate(ids):
    if id==19:
        fact = np.nanmean(crns6hc["2019-05-22":"2019-05-29"][4]) / np.nanmean(crns6hc["2019-05-22":"2019-05-29"][id])
        p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id]*fact, line_color=colors[i], line_width=1, legend=str(id))
    elif id==1:
        #continue
        fact2 = np.nanmean(crns6hc["2019-06-06 12:00:00":"2019-06-12 00:00:00"][1]) / np.nanmean(crns6hc["2019-05-22":"2019-05-29"][1])
        crns1 = crns6hc[1].copy()
        crns1[:"2019-06-06 03:00:00"] *= fact2
        fact = np.nanmean(crns6hc["2019-05-22":"2019-05-29"][4]) / np.nanmean(crns1["2019-05-22":"2019-05-29"])
        crns1 *= fact
        p.line(crns6hc.index+dt.timedelta(hours=3), crns1, line_color=colors[i], line_width=1, legend=str(id))
    else:
        fact = np.nanmean(crns6hc["2019-05-15":"2019-05-22"][4]) / np.nanmean(crns6hc["2019-05-15":"2019-05-22"][id])
        p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id]*fact, line_color=colors[i], line_width=1, legend=str(id))
    print(fact)
    if id==1:
        vline = Span(location=dt.datetime(2019,6,6,8,0,0), dimension='height', line_color='red', line_width=1, line_dash=[2])
        p.renderers.extend([vline])
        mytext = Label(x=dt.datetime(2019,6,6,8,0,0), y=1200, text='Shield rem.',
                       text_align="right", text_font_size="9pt")
        p.add_layout(mytext)
    p.xaxis.axis_label = 'Datetime'
    p.yaxis.axis_label = 'Counts per hour'
    #p.circle(crns[id].index, crns[id].cph1_filtered, size=1, color="lightgrey", legend="20 mins")
    #p.line(crns6h.index+dt.timedelta(hours=3), crns6h[id], line_color="grey", line_width=2, legend="uncorrected")
    #p.line(prec["2019-05-06":].index, prec["2019-05-06":].depth.cumsum(), line_color="blue", line_width=2, legend="precipitation")
    #p.line(crns6hc.index+dt.timedelta(hours=3), crns6hc[id], line_color="black", line_width=2, legend="6 hours")
    #p.line(crns24h.index+dt.timedelta(hours=12), crns24h[id], line_color="green", line_dash=[2], line_width=2, legend="24 hours")
    p.y_range=Range1d(5000, 9000)
    p.x_range=Range1d(min_dtime, max_dtime)
    p.legend.location = "center_left"
    p.legend.background_fill_alpha = 0.6
    #p.legend.location = "center_left"
    plts.append(p)

p.extra_y_ranges = {
    "Precipitation": Range1d(
        0, 250
    )
}
p.add_layout(LinearAxis(y_range_name="Precipitation"), "right")

p.line(prec["2019-05-06":].index, prec["2019-05-06":].depth.cumsum(), y_range_name="Precipitation", 
       line_color="black", line_width=2, line_dash = [2], legend="Prec.)")
output_file(htmlfile_merge, title="Time series of counts per hour")
#save(grid)
show(p)

# Add correct header to html
f = open(htmlfile_merge, "r")
html = f.read()
f.close()
html = html.strip()
replacement = '''<html>
<div class="blurb">
	<h1>JFC Fendt: Processed neutron counts  </h1>
</div><!-- /.blurb -->
<p> Data merged from SD and telemetry...counts per hour...filtered...resampled. 
'''
html = html.replace('<html lang="en">', replacement)
f = open(htmlfile_merge, "w")
f.write("---\nlayout: default\ntitle: cosmic pages\n---\n")
f.write(html)
f.close()

In [ ]:
crns6hc["2019-05-15":"2019-06-22"]

## From corrected neutron counts to soil moisture estimation

Volumetric soil moisture is estimated from a single relation that includes a local calibration parameter $N_0$. [Desilets et al. (2010)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2009WR008726) suggested the following one:

\begin{equation*}
\theta = \Bigl(\frac{a_0}{\frac{N_{corr}}{N_0} - a_1} - a_2\Bigl) * \frac{\rho_b}{\rho_w}
\end{equation*}

While $a_0$, $a_1$ and $a_2$ are empirical parameters, soil bulk density $\rho_b$ can be measured or retrieved via transfer functions or literature, the density of water $\rho_w$ is a function of temperature and salinity, although the approximate value of 1000 kg/m3 is can be used without introducing much error as compared to other uncertainties.

The parameter $N_0$ depends on the local hydrogen pool and footprint, and needs to be obtained from calibration with gravimetric soil moisture measurements in the footprint at a defined point in time. For the time being, we just take a wild guess and set $N_0$ to 1500 cph.

In [ ]:
ax = cs.core.n_to_theta_desilets(crns6hc, n0=1500.).plot(figsize=(14,12), grid=True, subplots=True)
for ax_ in ax:
    ax_.set_ylabel("theta (m3/m3)")
    ax_.set_ylim(-0.1, 1)

## Local calibration of $N_0$

The gravimetric soil moisture measurements are organised in a `DataFrame`.

In [ ]:
samples = pd.read_csv("../../../data/crns_sample1_soil.txt", sep=";")
samples.head()

In [ ]:
vol_cyl = 98. # cm3
samples["theta"] = (samples.wet_cyl_lab_g - samples.dry_cyl_lab_g) / vol_cyl
samples["rho_b"] = (samples.dry_cyl_lab_g - samples.cyl_g) / vol_cyl
samples.depth = [depth.strip() for depth in samples.depth]
samples["depth2"] = np.array([float(depth.split("-")[0]) for depth in samples.depth])
samples.head()

The sampling design is described in [Rivera Villarreyes, 2013](https://publishup.uni-potsdam.de/opus4-ubp/frontdoor/deliver/index/docId/6730/file/rivera_villarreyes_diss.pdf), p. 104. Letter `A-F` refer to the six angles, numbers `1-4` to the distances `25, 75, 175, and 225` respectively. We assign the distannce of the measurements to the CRNS probe on that basis, assuming that "Center" means a distance of 1 m.

In [ ]:
samples["r"] = np.nan
for i in range(len(samples)):
    if "1" in samples.location[i]:
        samples.loc[i, "r"] = 25.
    elif "2" in samples.location[i]:
        samples.loc[i, "r"] = 75.
    elif "3" in samples.location[i]:
        samples.loc[i, "r"] = 175.
    elif "4" in samples.location[i]:
        samples.loc[i, "r"] = 225.
    elif samples.location[i]=="Center":
        samples.loc[i, "r"] = 1.

In [ ]:
plt.figure(figsize=(6,5))
ax = plt.subplot(111)
for loc in np.unique(samples.location):
    depths = samples[samples.location==loc].depth2
    theta = samples[samples.location==loc].theta
    #print(theta, depths)
    plt.plot(theta, -depths-2.5, label="_exclude")
avg = samples.groupby("depth2").mean()
plt.plot(avg.theta, -avg.index-2.5, linewidth=4, color="black", label="Mean profile")
plt.xlabel("Theta (m3/m3)")
plt.ylabel("Depth (cm)")
plt.title("Measured theta profiles and average profile")
plt.grid()
leg = plt.legend()

Schroen et al. (2017) suggest a five step procedure for obtaining the average soil moisture `theta_avg` in the CRNS footprint:

1. **Initial guess:** obtain initial estimate of `theta_avg` (e.g. as the simple mean of all measurements)
2. **Penetration depth:** compute the penetration depth for each profile, depending on distance `r`, `theta_avg`, as well as barometric pressure `press`, vegetation height `Hveg`, and bulk density `rho_b`.
3. **Vertical averaging:** compute the average soil moisture for each measured profile, based on vertical weights for each profile (implicitely requires penetration depth).
4. **Horizontal averaging:** compute `theta_avg` by as a weighted average, obtaining horizontal weights as a function of distance `r`, `theta_avg`, as well as barometric pressure `press`, vegetation height `Hveg`, and absolute humidity `h`.
5. Iterate over 1-4 until `theta_avg` converges to a user-defined accuracy.

Yet, the approach to first apply vertical averaging and *then* horizontal averaging only makes sense if the sampling is identical for each profile. E.g., if we had six samples per profile, but one profil with only *one* sample, that last profile would be overemphasized if we would first average on a per-profile basis. It would thus be more adequate to apply vertical and horizontal weighting simultaneously. Beyond, *step 2* of the above procedure could be considered as part of *step 3*, since the penetration depth is required to obtain the vertical weights.

On the basis, we suggest the following revised procedure:

1. Initial guess
2. Vertical and horizontal averaging
3. Iteration

#### Step 1: initial guess

Initial guess of $\theta$ is the average of all samples.

In [ ]:
theta_avg = samples.theta.mean()

#### Step 2: penetration depth

In [ ]:
# Set parameters which are asssumed to be uniform across the footprint
rho_b = samples.rho_b.mean()
press = crns["tower"]["2013-08-05"].press.mean()
h = np.mean(cs.conv.absolute_humidity(crns["tower"]["2013-08-05"].temp1, crns["tower"]["2013-08-05"].relhum1) )
Hveg = 0.

In [ ]:
D_p = cs.core.D86(samples.r, theta=theta_avg, press=press, Hveg=Hveg, rhob=rho_b)

#### Step 3: vertical averaging per profile

In [ ]:
profs = pd.DataFrame({}, index=np.unique(samples.location) )
profs["theta"] = np.nan
profs["r"] = np.nan
for prof in profs.index:
    ix = samples.location==prof
    profs.loc[prof, "r"] = np.unique(samples[ix].r)[0]
    vweights = cs.core.vertical_weight_koehli(profs.loc[prof, "r"], samples[ix].depth2, theta_avg, press, Hveg, rho_b)
    profs.loc[prof, "theta"] = np.sum(vweights * samples[ix].theta) / np.sum(vweights)

#### Step 4: horizontal averaging

In [ ]:
hweights = cs.core.horizontal_weight_koehli(profs.r, press, Hveg, theta_avg, h)
theta_avg = np.sum(hweights * profs.theta) / np.sum(hweights)

#### Step 5: iterate until `theta_avg` converges 

In [ ]:
max_iter = 5
theta_avg = samples.theta.mean()
for i in range(max_iter):
    print(i, np.round(theta_avg,3))
    # step 2
    D_p = cs.core.D86(samples.r, theta=theta_avg, press=press, Hveg=Hveg, rhob=rho_b)
    # step 3
    profs = pd.DataFrame({}, index=np.unique(samples.location) )
    profs["theta"] = np.nan
    profs["r"] = np.nan
    for prof in profs.index:
        ix = samples.location==prof
        profs.loc[prof, "r"] = np.unique(samples[ix].r)[0]
        vweights = cs.core.vertical_weight_koehli(profs.loc[prof, "r"], samples[ix].depth2, theta_avg, press, Hveg, rho_b)
        profs.loc[prof, "theta"] = np.sum(vweights * samples[ix].theta) / np.sum(vweights)
    # step 4
    hweights = cs.core.horizontal_weight_koehli(profs.r, press, Hveg, theta_avg, h)
    theta_avg = np.sum(hweights * profs.theta) / np.sum(hweights)
    


With only one single set of soil moisture measurements for a single point in time, calibration only involves solving the relationship between $\theta (N)$ for $N_0$:

\begin{equation*}
N_0 = N * \frac{\theta * \frac{\rho_w}{\rho_b} + a_2}{a_0 + \theta * \frac{\rho_w}{\rho_b} * a_1 + a_1 * a_2}
\end{equation*}

In [ ]:
def calibrate_n0_desilets(theta, cph, a0=0.0808, a1=0.372, a2=0.115, rhob=1500., rhow=1000.):
    """
    """
    return cph * (theta*(rhow/rhob) + a2) / (a0 + theta*(rhow/rhob) * a1 + a1 * a2)

In [ ]:
cph = crns6hc["2013-08-05 00:00:00":"2013-08-06 00:00:00"].tower.mean()
n0 = calibrate_n0_desilets(theta_avg, cph)
print("N0 =", n0)

In [ ]:
theta = cs.core.n_to_theta_desilets(crns6hc.resample("168H").mean(), n0=n0)

In [ ]:
ax = theta.tower.plot(figsize=(14,12), grid=True, subplots=True)
for ax_ in ax:
    ax_.set_ylabel("theta (m3/m3)")
    ax_.set_ylim(-0.1, 1)